# Logging

## Sources

- <a href="https://habr.com/ru/companies/wunderfund/articles/683880/">Logging in python: developer guide (rus)</a>;

In [7]:
import os
import sys
import logging
import random

## Error types

There are five levels of logging:

- Debug;
- Info;
- Warning;
- Error;
- Critical.

You can call them by using the relevant functions:

In [3]:
logging.debug("A DEBUG Message")
logging.info("An INFO")
logging.warning("A WARNING")
logging.error("An ERROR")
logging.critical("A message of CRITICAL severity")

ERROR:root:An ERROR
CRITICAL:root:A message of CRITICAL severity


In previous examle `debug` and `info` errors wasn't printed. It happened becaulse default value for `logging.level` setted to `logging.WARNING`, therefore only those messages above `WARNING` will be displayed. 

## Basic logger

This logger is used when you run the logging methods of the `logging` module itself.

Using the `logging.basicConfig` function, you can define the behaviour of the logs generated by the `logging` module. The following example defines the `DEBUG` logging level so that you can later see any message raised by the `logging` methods. By default only `warning`, `error` and `critical` are executed.

**Note** In the following example, the code is saving the logs to a separate file and executing it from another interpreter. This approach is necessary because once you execute any logging method in this notebook, you cannot modify its configuration anymore.

In [4]:
%%writefile logging_files/basic_logger.py
import logging
logging.basicConfig(level=logging.DEBUG)

logging.debug("A DEBUG Message")
logging.info("An INFO")
logging.warning("A WARNING")
logging.error("An ERROR")
logging.critical("A message of CRITICAL severity")

Overwriting logging_files/basic_logger.py


In [5]:
!python3 logging_files/basic_logger.py

DEBUG:root:A DEBUG Message
INFO:root:An INFO
ERROR:root:An ERROR
CRITICAL:root:A message of CRITICAL severity


## Loggers

For different parts of the programs you'll need to have defferent `loggers`. So you can create new loger by using `logging.getLogger(<loger name>)`.

In [8]:
logger1 = logging.getLogger("logger1")
logger1.setLevel(logging.INFO)
print("=====logger1=====", file=sys.stderr)
logger1.debug("A DEBUG Message")
logger1.info("An INFO")
logger1.warning("A WARNING")
logger1.error("An ERROR")
logger1.critical("A message of CRITICAL severity")


logger2 = logging.getLogger("logger2")
logger2.setLevel(logging.ERROR)
print("=====logger2=====", file=sys.stderr)
logger2.debug("A DEBUG Message")
logger2.info("An INFO")
logger2.warning("A WARNING")
logger2.error("An ERROR")
logger2.critical("A message of CRITICAL severity")

=====logger1=====
INFO:logger1:An INFO
ERROR:logger1:An ERROR
CRITICAL:logger1:A message of CRITICAL severity
=====logger2=====
ERROR:logger2:An ERROR
CRITICAL:logger2:A message of CRITICAL severity
